In [2]:
import pandas as pd
import numpy as np

# READ THE CSV INTO DATAFRAME

df = pd.read_csv('Syngenta/Syngenta_2017/Experiment_dataset.csv')

# np_ar = np.asarray(df)
# df2 = pd.read_csv('Syngenta/Syngenta_2017/Region_dataset.csv')

In [3]:
print(df['Planting date'][[np.random.randn]])
# print(df.columns)
# print(df.Temperature.describe())
# import random
# for i in range(0, 500):
#     print(random.choice(df['Planting date']))

KeyError: 'None of [[<built-in method randn of mtrand.RandomState object at 0x7f6cc00eb048>]] are in the [index]'

In [ ]:
df.Yield.describe()

In [6]:
# CURRENTLY NECESSARY IF: USING 174 ADDITIONAL VARIETY COLUMNS METHOD

# THIS IS A DIFFERENT APPROACH TO THE ABOVE FOUR CELLS, WHERE WE HAVE 174 ADDITIONAL FEATURE COLUMNS
# EACH WITH A 0 (IF IT IS NOT OF THAT VARIETY) OR A 1 (IF IT IS OF THAT VARIETY)

# print(df)
variety_dummies = pd.get_dummies(df.Variety)
# print(dummies)
df = pd.concat([df, variety_dummies], axis=1)


In [ ]:
# OPTIONAL VARIETY DISTRIBUTION ANALYSIS

print(variety_dummies.sum().describe())
print(np.sort(variety_dummies.sum()))
for idx, cl in enumerate(variety_dummies.sum()):
    print(variety_dummies.columns[idx], cl)

In [ ]:
# GOAL OF THIS MODULE:
# Encode the planting date as a season
# NEW GOAL:
# GET DUMMIES FOR SEASONS

# remove the dates that are "."
df = df[~df['Planting date'].str.match("\.")]
plant_date = df['Planting date'].apply(lambda dt: pd.to_datetime(dt))
plant_months = plant_date.apply(lambda dt: dt.month)
season = plant_date.rename("Season")
season = pd.to_datetime(season)
season = season.apply(lambda dt: (dt.month%12 + 3)//3)
# df['Plant date'] = pd.to_datetime(df['Plant date'])
df = pd.concat([df, season], axis=1)

# plant_date = pd.to_datetime(df['Planting date'], infer_datetime_format=True)
# df = df['Planting date'].apply(lambda dt: (dt.month%12 + 3)//3)
# pd.get_dummies(df['Planting date'])


In [ ]:
# ADD MONTH OF MAY AND JUNE ONE HOT ENCODING INTO THE DATAFRAME
pd.get_dummies(plant_months).sum()
june = pd.get_dummies(plant_months).loc[:,6]
june = june.rename("June")
may = pd.get_dummies(plant_months).loc[:,5]
may = may.rename("May")
df = pd.concat([df, may], axis=1)
df = pd.concat([df, june], axis=1)

In [ ]:
print(df.columns)

In [ ]:
pd.get_dummies(df['Season']).sum()

In [ ]:
print(df.columns)
print(df.isnull().sum())

In [ ]:
# LATITUDE AND LONGITUDE CLUSTERING INTO FEATURES

from sklearn.cluster import KMeans

latlong = df.loc[:, ['Latitude', 'Longitude']]

kmeans = KMeans(n_clusters=4, random_state=0).fit(latlong)
kmeans.labels_.shape
lat_long_dummies = pd.get_dummies(kmeans.labels_)
lat_long_dummies = lat_long_dummies.rename(index=int, columns={0: "Loc Clust 0",
                                                               1: "Loc Clust 1",
                                                               2: "Loc Clust 2",
                                                               3: "Loc Clust 3"})
df = pd.concat([df, lat_long_dummies], axis = 1)

In [ ]:
np.asarray(df.iloc[:, df.columns.str.match('V\d\d\d\d\d\d')].columns)

In [ ]:
#THIS IS A VISUALIZATION FOR LATITUDE AND LONGITUDE CLUSTERING

cent = kmeans.cluster_centers_
clust_labels = kmeans.labels_
means = pd.DataFrame(clust_labels)

import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_subplot(111)
scatter = ax.scatter(df['Latitude'],df['Longitude'],
                     c=means[0], s=50)
ax.set_title('K-Means Clustering')
ax.set_xlabel('Latitude')
ax.set_ylabel('Longitude')
plt.colorbar(scatter)

In [ ]:
#REMOVE ANY NAN VALUES

print(df.columns)
df = df[~df.Silt.isnull()]
df = df[~df['Loc Clust 1'].isnull()]

In [ ]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
binarized = lb.fit(df.Variety)
print(binarized)
df.Variety = pd.Series(binarized.transform(df.Variety))
print(binarized.transform(df.Variety).reshape(1,-1))

In [ ]:
for col in df.columns:
    print(col, type(df[col][0]))

In [7]:
# DROP ALL THE CELLS THAT ARE NOT USABLE SUCH AS THE ONES THAT ARE STRINGS OR DATES

# set if want to drop some columns specifically
should_drop = 1
columns_to_drop = ['Experiment', 'Location',
                   'Check Yield', 'Yield difference', 'Latitude',
                   'Longitude', 'Variety', 'PI', 'Planting date']

# set if want to keep some columns specifically
should_keep = 0
# columns_to_keep = ['Loc Clust 0', 'Loc Clust 1', 'Loc Clust 2', 'Loc Clust 3']
columns_to_keep_top = ['Silt', 'Precipitation', 'Temperature', 'Solar Radiation', 'Organic matter']
columns_VARIETIES_ONLY = np.asarray(df.iloc[:, df.columns.str.match('V\d\d\d\d\d\d')].columns)

#set the below variable to whatever columns you want to keep
columns_to_keep = columns_to_keep_top

MUST_HAVE_COLUMNS = ['Yield']
# print(columns_to_keep)

df = df.drop(columns_to_drop, axis=1) if should_drop else df
df = df.loc[:, np.concatenate((columns_to_keep, MUST_HAVE_COLUMNS))] if should_keep else df
df['YieldBucket'] = pd.Series(pd.qcut(df.Yield, q=3, labels=["high", "medium", "low"]))
print("The final dataframe has columns: ", df.columns)

The final dataframe has columns:  Index(['Yield', 'Year', 'Temperature', 'Precipitation', 'Solar Radiation',
       'Soil class', 'CEC', 'Organic matter', 'pH', 'Clay',
       ...
       'V156574', 'V156642', 'V156763', 'V156774', 'V156783', 'V156786',
       'V156797', 'V156806', 'V156807', 'YieldBucket'],
      dtype='object', length=188)


In [ ]:
print(pd.get_dummies(df.YieldBucket).sum())

In [ ]:
# LET US ALSO MAKE SURE THERE ARE NO NAN IN THE DATA
print("We expect to be %s nan values and there actually are %s nan values\n" % (0, np.sum(df.isnull().sum())))
print(df.isnull().sum())
# AFTER COLUMNS, MAKE SURE NO SKETCHY ONES
for col in df.columns:
    print(col, type(df[col][0]))    

In [8]:
# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT
# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT
# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT
# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT

X = df.drop(['Yield', 'YieldBucket'], axis=1)

print(X.columns)

y = df.Yield

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.05, train_size = 0.1, random_state = 42)

INPUT_COLS = X_train.columns
# TEST_COLS = y_train.columns

Index(['Year', 'Temperature', 'Precipitation', 'Solar Radiation', 'Soil class',
       'CEC', 'Organic matter', 'pH', 'Clay', 'Silt',
       ...
       'V156565', 'V156574', 'V156642', 'V156763', 'V156774', 'V156783',
       'V156786', 'V156797', 'V156806', 'V156807'],
      dtype='object', length=186)


In [5]:
print("X_train shape:", X_train.shape, "\ny_train shape:", y_train.shape)

X_train shape: (8203, 186) 
y_train shape: (8203,)


In [ ]:
# This function will evaluate the errors based on RMSE (from the challenge spec)
# also will evaluate based on average error

from sklearn.metrics import mean_squared_error
def evaluate_errors(prediction, actual):
    print("RMSE Error: ", np.sqrt(mean_squared_error(prediction, actual)))
    avg_error_vector = np.absolute(((preds - y_test) / y_test) * 100)
#     print("Average Error: ", np.mean(avg_error_vector))
    print("Average Error details:\n", avg_error_vector.describe())
    return avg_error_vector

In [ ]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(n_estimators=20, max_depth=13, random_state=0, verbose=1)
regr.fit(X_train, y_train)
preds = regr.predict(X_test)

evaluate_errors(preds, y_test)


In [ ]:
pd.get_dummies(df['Plant date']).sum()

In [ ]:
# GET OUTPUT OF FEATURE IMPORTANCE
def get_feature_importances(regr):
    feature_importances = regr.feature_importances_
    feature_importances = pd.Series(feature_importances)
    feature_importance_df = pd.DataFrame({'feature': X_train.columns,'feature_importance': feature_importances})
    feature_importance_df = feature_importance_df.sort_values(by=['feature_importance'])
    for index, row in feature_importance_df.iterrows():
        print(row['feature'], 'has importance: ', row['feature_importance'])
get_feature_importances(regr)

In [ ]:
# THIS WILL ONLY WORK WITH THE BUCKET METHOD

from sklearn.ensemble import RandomForestClassifier
regr = RandomForestClassifier(n_estimators=10, max_depth=20, random_state=0, verbose=1)
regr.fit(X_train, y_train)
preds = regr.predict(X_test)

from sklearn.metrics import accuracy_score

print(accuracy_score(y_test, preds))

In [ ]:
import numpy as np
from sklearn import linear_model
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPRegressor

from sklearn.feature_selection import RFECV

classifiers = [
    svm.SVR(),
    MLPRegressor(solver='lbfgs', alpha=1e-5,
                     hidden_layer_sizes=(5, 2), random_state=1),
    linear_model.SGDRegressor(),
    linear_model.BayesianRidge(),
    linear_model.LassoLars(),
#     linear_model.ARDRegression(),
#     linear_model.ARDRegression(),
    linear_model.PassiveAggressiveRegressor(),
    linear_model.TheilSenRegressor(),
    linear_model.LinearRegression()]




# estimator = svm.SVR(kernel="linear")

# selector = RFECV(estimator, step=1, cv=5, verbose=1)
# selector = selector.fit(X_train, y_train)
# selector.support_ 
# # array([ True,  True,  True,  True,  True,
# #         False, False, False, False, False], dtype=bool)
# selector.ranking_
# # array([1, 1, 1, 1, 1, 6, 4, 3, 2, 5])


#     print(np.sum(preds - y_test))
#     print(clf.predict(X_test),'\n')
#     print(y_test)
#     print('accuracy score:', accuracy_score(y_test, clf.predict(X_test)), '\n')


In [ ]:
for item in classifiers:
    print(item)
    clf = item
    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)
    errors = evaluate_errors(preds, y_test)
    try:
        get_feature_importances(clf)
    except:
        print("NO FEATURE IMPORTANCE METRIC")
#     errors = np.absolute(((preds - y_test) / y_test) * 100)
#     print(errors)
#     print(errors)
#     print(errors.describe())

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

classifiers = [
#     KNeighborsClassifier(3),
#     SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
#     GaussianProcessClassifier(1.0 * RBF(1.0)),
#     DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]
from sklearn.metrics import accuracy_score
for item in classifiers:
    print(item)
    clf = item
    clf.fit(scale(X_train), y_train)
    preds = clf.predict(scale(X_test))
    print(accuracy_score(y_test, preds))
#     errors = np.absolute(((preds - y_test) / y_test) * 100)
#     print(errors)
#     print(np.mean(errors))

In [42]:
import random
duplicated_df = pd.DataFrame()
counter = 0
X_test_random_rows = X_test.sample(n=174)
# print(X_test_random_rows)
for index, row in X_test_random_rows.iterrows():
    counter+=1
    print(counter)
    duplicated_df = duplicated_df.append([row]*174, ignore_index=True)
#     print([row]*2)
#     break
#     print(X_test[row.name])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174


In [43]:
duplicated_df_varieties = duplicated_df.loc[:, duplicated_df.columns.str.match('V\d\d\d\d\d\d')]

In [44]:
duplicated_df_varieties.shape

(30276, 174)

In [62]:
ddv = duplicated_df_varieties

In [70]:
d = np.zeros((30276, 174))

In [71]:

duplicated_df_varieties = pd.DataFrame(d, columns = duplicated_df_varieties.columns)

In [73]:
varieties_array = duplicated_df_varieties.columns

In [74]:
print(varieties_array.shape)

(174,)


In [76]:
ddv = duplicated_df_varieties

In [84]:
ddv.loc[0, varieties_array[0]] = 1
# varieties_array[3]
ddv.sum()

V000016    0.0
V000017    0.0
V000018    0.0
V000023    0.0
V000024    0.0
V000025    0.0
V000030    0.0
V000032    0.0
V000034    0.0
V000036    0.0
V000039    0.0
V000047    0.0
V000050    0.0
V000051    0.0
V000058    0.0
V000060    0.0
V000062    0.0
V000067    0.0
V000070    0.0
V000071    0.0
V000075    0.0
V000078    0.0
V000079    0.0
V000080    0.0
V000081    0.0
V000082    0.0
V000092    0.0
V000096    0.0
V000098    0.0
V000110    0.0
          ... 
V152253    0.0
V152300    0.0
V152312    0.0
V152320    0.0
V152322    0.0
V152440    0.0
V152734    0.0
V152779    0.0
V155180    0.0
V155820    0.0
V155842    0.0
V155843    0.0
V155918    0.0
V156247    0.0
V156305    0.0
V156314    0.0
V156367    0.0
V156368    0.0
V156516    0.0
V156553    0.0
V156565    0.0
V156574    0.0
V156642    0.0
V156763    0.0
V156774    0.0
V156783    0.0
V156786    0.0
V156797    0.0
V156806    0.0
V156807    0.0
Length: 174, dtype: float64

In [72]:
duplicated_df_varieties.sum()

for i in range(duplicated_df_varieties.shape[0]):
    var_index = i % 174
    duplicated_df_varieties.loc[i, varieties_array[var_index]]

V000016    0.0
V000017    0.0
V000018    0.0
V000023    0.0
V000024    0.0
V000025    0.0
V000030    0.0
V000032    0.0
V000034    0.0
V000036    0.0
V000039    0.0
V000047    0.0
V000050    0.0
V000051    0.0
V000058    0.0
V000060    0.0
V000062    0.0
V000067    0.0
V000070    0.0
V000071    0.0
V000075    0.0
V000078    0.0
V000079    0.0
V000080    0.0
V000081    0.0
V000082    0.0
V000092    0.0
V000096    0.0
V000098    0.0
V000110    0.0
          ... 
V152253    0.0
V152300    0.0
V152312    0.0
V152320    0.0
V152322    0.0
V152440    0.0
V152734    0.0
V152779    0.0
V155180    0.0
V155820    0.0
V155842    0.0
V155843    0.0
V155918    0.0
V156247    0.0
V156305    0.0
V156314    0.0
V156367    0.0
V156368    0.0
V156516    0.0
V156553    0.0
V156565    0.0
V156574    0.0
V156642    0.0
V156763    0.0
V156774    0.0
V156783    0.0
V156786    0.0
V156797    0.0
V156806    0.0
V156807    0.0
Length: 174, dtype: float64

In [ ]:

def yield_per_variety(clf, env_conditions):
    